# Monte-Carlo control 

We start by importing gym. The environment we will use is text-based, so there is no need for all the boilerplate of last exercise: we simply pip install gym if we are on Colab.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng()
import time

from IPython.display import clear_output

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install gym > /dev/null 2>&1
    
import gym

## The taxi environment

In this exercise, we are going to apply **on-policy Monte-Carlo control** on the Taxi environment available in gym:

<https://gym.openai.com/envs/Taxi-v3/>

In [2]:
import gym.envs.toy_text.taxi
help(gym.envs.toy_text.taxi)

Help on module gym.envs.toy_text.taxi in gym.envs.toy_text:

NAME
    gym.envs.toy_text.taxi

CLASSES
    gym.envs.toy_text.discrete.DiscreteEnv(gym.core.Env)
        TaxiEnv
    
    class TaxiEnv(gym.envs.toy_text.discrete.DiscreteEnv)
     |  The Taxi Problem
     |  from "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"
     |  by Tom Dietterich
     |  
     |  Description:
     |  There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.
     |  
     |  Observations: 
     |  There are 500 discrete states since there are 25 taxi positions, 5 possible locations 

Let's create the environment, initialize it  and render the first state:

In [3]:
env = gym.make("Taxi-v3")
state = env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The agent is the yellow square. It can move up, down, left or right if there is no wall (the pipes and dashes). Its goal is to pick clients at the blue location and drop them off at the pink location. These locations are fixed (R, G, B, Y), but which one is the pick-up location and which one is the drop-off destination changes between each episode.

**Q:** Re-run the previous cell multiple times to observe the diversity of initial states.

The following cell prints the action space of the environment: 

In [4]:
print("Action Space", env.action_space)
print("Number of actions", env.action_space.n)

Action Space Discrete(6)
Number of actions 6


There are 6 discrete actions: south, north, east, west, pickup, dropoff.
    
Let's now look at the observation space (state space):

In [5]:
print("State Space", env.observation_space)
print("Number of states", env.observation_space.n)

State Space Discrete(500)
Number of states 500


There are 500 discrete states. What are they?

* The taxi can be anywhere in the 5x5 grid, giving 25 different locations.
* The passenger can be at any of the four locations R, G, B, Y or in the taxi: 5 values.
* The destination can be any of the four locations: 4 values.

This gives indeed 25x5x4 = 500 different combinations.

The internal representation of a state is a number between 0 and 499. You can use the `encode` and `decode` methods of the environment to relate it to the state variables.

In [6]:
state = env.encode(2, 1, 1, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

state = env.decode(328) 
print("State:", list(state))

State: 224
State: [3, 1, 2, 0]


The reward function is simple:

* r = 20 when delivering the client at the correct location.
* r = -10 when picking or dropping a client illegally (picking where there is no client, dropping a client somewhere else, etc)
* r = -1 for all other transitions in order to incent the agent to be as fast as possible.

The actions pickup and dropoff are very dangerous: take them at the wrong time and your return will be very low. The navigation actions are less critical.

Depending on the initial state, the taxi will need at least 10 steps to deliver the client, so the maximal return you can expect is around 10 (+20 for the success, -1 for all the steps). 

The task is episodic: if you have not delivered the client within 200 steps, the episode stops (no particular reward).

## Random agent

Let's now define a random agent that just samples the action space.

**Q:** Modify the random agent of last time, so that the agent performs a fixed number of **episodes**, not steps. Make sure to use the `done` flag to break the for loop and start in a new state. Optionally render the state of the agent at every step. At the end of each episode, compute its return using $\gamma = 1.0$ (i.e. simply sum the obtained rewards) and print the list in the end.

*Tip:* If you render the state at every step, they will be printed one after the other. To have an animation, you can clear the output of the cell using:

```python
from IPython.display import clear_output # already imported

for t in episode: 
    # ...
    clear_output(wait=True)
    env.render()
    # ...
```

*Tip:* The animation may be too fast to visualize anything. To force the framerate to be low enough, you can make Python "sleep" (do nothing) for a few milliseconds after each rendering:

```python
import time # already imported

for t in episode: 
    clear_output(wait=True)
    env.render()
    time.sleep(0.1) # sleep for 100 milliseconds
```

**Q:** What do you think of the returns obtained by the random agent? Conclude on the difficulty of the task.

## On-policy Monte-Carlo control

Now let's apply on-policy MC control on the Taxi environment. As a reminder, here the meta-algorithm:

* **while** True:

    1. Generate an episode $\tau = (s_0, a_0, r_1, \ldots, s_T)$ using the current **stochastic** policy $\pi$.

    2. For each state-action pair $(s_t, a_t)$ in the episode, update the estimated Q-value:

    $$
        Q(s_t, a_t) = Q(s_t, a_t) + \alpha \, (R_t - Q(s_t, a_t))
    $$

    3. For each state $s_t$ in the episode, improve the policy (e.g. $\epsilon$-greedy):

    $$
        \pi(s_t, a) = \begin{cases}
                        1 - \epsilon \; \text{if} \; a = a^* \\
                        \frac{\epsilon}{|\mathcal{A(s_t)}-1|} \; \text{otherwise.} \\
                      \end{cases}
    $$
    
In practice, we will need:

* a **Q-table** storing the estimated Q-value of each state-action pair: its size will be (500, 6).

* an $\epsilon$-greedy action selection to select actions in the current state.

* an learning mechanism allowing to update the Q-value of all state-action pairs encountered in the episode.

**Q:** Create a `MonteCarloAgent` agent implementing on-policy MC for the Taxi environment. Use $\gamma = 0.9$, $\epsilon = 0.1$ and $\alpha=0.01$ (pass these parameters to the constructor of the agent and store them). Train the agent for 20000 episodes (yes, 20000... Start with one episode to debug everything and then launch the simulation. It should take around one minute). Save the return of each episode in a list and plot them in the end. 

Implementing the action selection should not be a problem, it is the same as for bandits. Little trick (not obligatory): you can implement $\epsilon$-greedy as:

```python
action = self.Q[state, :].argmax()
if rng.random() < epsilon:
    action = self.env.action_space.sample()
```

This is not exactly $\epsilon$-greedy, as `env.action_space.sample()` may select the greedy action again. In practice it does not matter, it only changes the meaning of $\epsilon$, but the action selection stays similar. It is better to rely on `env.action_space.sample()` for the exploration, as some Gym problem work better with a normal distribution for the exploration than with uniform (e.g. continuous problems). 

Do not select the greedy action with `self.Q[state, :].argmax()` but `rng.random.choice(np.where(self.Q[state, :] == self.Q[state, :].max())[0])`: at the beginning of learning, where the Q-values are all 0, you would otherwise always take the first action (south).

The `update()` method should take a complete episode as argument, using a list of (state, action, reward) transitions. It should be called at the end of an episode only, not after every step.

A bit tricky is the calculation of the returns for each visited state. The naive approach would look like:

```python
T = len(episode)
for t in range(T):
    state, action, reward = episode[t]
    return_state = 0.0
    for k in range(t, T): # rewards coming after t
        next_state, next_action, next_reward = episode[k]
        return_state += gamma**k * reward
    self.Q[state, action] += alpha * (return_state - self.Q[state, action])
```

The double for loop can be computationally expensive for long episodes (complexity T log T). It is much more efficient to iterate **backwards** on the episode, starting from the last transition and iterating until the first one, and using the fact that:

$$R_{t} = r_{t+1} + \gamma \, R_{t+1}$$

The terminal state $s_T$ has a return of 0 by definition. The last transition $s_{T-1} \rightarrow s_{T}$ has therefore a return of $R_{T-1} = r_T$. The transition before that has a return of $R_{T-2} = r_{T-1}  + \gamma \, R_{T-1}$, and so on. You can then compute the returns of each action taken in the episode (and update its Q-value) in **linear time**.

To iterate backwards over the list of transitions, use the `reversed()` operator:

```python
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for a in reversed(l):
    print(a)
```

As you may observe, the returns have a huge variance due to the exploration, what makes the plot quite ugly and unreadable. The following function allows to smooth the returns using a sliding average over the last $N$ epochs:

In [ ]:
def running_average(x, N):
    cumsum = np.cumsum(np.insert(np.array(x), 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N

**Q:** Plot the returns as well as their sliding average. Comment on the influence of exploration. 

**Q:** Extend the agent with a `test(self, render)` method that performs a single episode on the environment without exploration, optionally renders each state but does **not** learn. You will have to re-train the agent, because the definition of its class has changed. Backup the previous value of `epsilon` in a temporary variable and reset it at the end of the episode. Have the method return the **undiscounted** sum of rewards in the episode, as well as the number of steps until termination.

After training (you can reduce the number of episodes), first observe a couple of test episodes with rendering on. Is the policy any good?

Perform 1000 test episodes without rendering and report the mean return over these 1000 episodes as the final performance of your agent.

## Experiments

### Early stopping

**Q:** Train the agent for the smallest number of episodes where the returns seem to have stabilized (e.g. 2000 episodes). Test the agent. Does it work? Why?

### Discount rate

**Q:** Change the value of the discount factor $\gamma$. As the task is episodic (maximum 200 steps), try a discount rate of 1. What happens? Conclude.

### Learning rate

**Q:** Vary the learning rate `alpha`. What happens?

### Exploration parameter

**Q:** Vary the exploration parameter `epsilon` and observe its impact on learning.

### Exploration scheduling

Even with a good learning rate (0.01) and a discount factor of 0.9, the exploration parameter as a huge impact on the performance: too low and the agent does not find the optimal policy, too high and the agent is inefficient at the end of learning. 

**Q:** Implement scheduling for epsilon. You can use exponential scheduling as in the bandits exercise:

$$\epsilon = \epsilon \times (1 - \epsilon_\text{decay})$$

at the end of each episode, with $\epsilon_\text{decay}$ being a small decay parameter (`1e-5` or so).

Find a correct value for $\epsilon_\text{decay}$. Do not hesitate to fine-tune alpha at the same time.

*Tip:* Prepare and visualize the scheduling in a different cell, and use the initial value of $\epsilon$ and $\epsilon_\text{decay}$ that seem to make sense. 